# Simulador de inserciones en vivo — Salmonera (SQL Server / SQLEXPRESS)

Este notebook crea el esquema, siembra dimensiones y ejecuta inserciones continuas (con datos sucios) en todas las tablas: clientes, productos, empleados, producción, ventas, exportaciones, costos e inventario.

**Ajusta la conexión** en la siguiente celda para tu `SQLEXPRESS` (Windows auth o SQL auth).

In [1]:
# ▶️ 1) (Opcional) Instalar librerías desde el Notebook
!pip install pyodbc python-dateutil tqdm

In [2]:
# ▶️ 2) Configuración de conexión — SQLEXPRESS
import os

SERVER   = "localhost"         # o IP/hostname
DATABASE = "SalmonesBD"         # crea la BD antes o usa master y cambia USE
USERNAME = "sa"
PASSWORD = "hola123"
DRIVER   = "ODBC Driver 18 for SQL Server"  # nombre exacto del driver
TRUSTCERT = "yes"  # 'yes' si usas Encrypt por defecto en Driver 18

import pyodbc

DRIVER   = "ODBC Driver 18 for SQL Server"
SERVER   = r"localhost\SQLEXPRESS"      # o r"TU-PC\SQLEXPRESS"
DATABASE = "SalmonesBD"                  # crea la BD si aún no existe
AUTH     = "windows"                    # "windows" o "sql"

USERNAME = "sa"                         # solo si AUTH="sql"
PASSWORD = "YourStrong!Passw0rd"        # solo si AUTH="sql"

def get_conn():
    if AUTH.lower() == "windows":
        conn_str = (
            f"DRIVER={{{DRIVER}}};SERVER={SERVER};DATABASE={DATABASE};"
            "Trusted_Connection=Yes;Encrypt=yes;TrustServerCertificate=yes;"
        )
    else:
        conn_str = (
            f"DRIVER={{{DRIVER}}};SERVER={SERVER};DATABASE={DATABASE};"
            f"UID={USERNAME};PWD={PASSWORD};Encrypt=yes;TrustServerCertificate=yes;"
        )
    return pyodbc.connect(conn_str, autocommit=True)

# Test
with get_conn() as conn:
    cur = conn.cursor()
    cur.execute("SELECT @@SERVERNAME, CAST(SERVERPROPERTY('InstanceName') AS NVARCHAR(100));")
    print("Conectado a:", cur.fetchone())

Conectado a: ('DESKTOP-J11EQUD\\SQLEXPRESS', 'SQLEXPRESS')


In [3]:
#Helpers robustos (previenen el error de IDs texto)
import random
from datetime import datetime, timedelta

PLANTAS = ["Chinquihue","Calbuco","Quellón","Hornopirén","Puerto Montt"]
ESPECIES = ["Atlántico","Coho","Trucha"]
PROCESOS = ["Fileteado","Congelado","Envasado","Despacho","Almacenado"]
TURNOS   = ["Mañana","Tarde","Noche"]
ESTADOS  = ["A tiempo","Retrasado","Cancelado"]
PAISES   = ["EEUU","China","Japón","Brasil","España"]
UBICS    = ["Bodega Norte","Bodega Sur","Puerto"]
CARGOS   = ["Operario","Supervisor","Administrativo","Técnico"]

P_NULL_PRICE = 0.02
P_NEG_KG     = 0.02
P_TYPO       = 0.01

def rand_dt(days_back=730):
    now = datetime.now()
    d = timedelta(days=random.randint(0, days_back),
                  hours=random.randint(0,23),
                  minutes=random.randint(0,59))
    return now - d

def maybe_typo(s):
    return s.replace("o","0").replace("a","@") if random.random()<P_TYPO else s

def get_next_id(cur, table, id_col):
    cur.execute(f"SELECT CAST(ISNULL(MAX(TRY_CAST({id_col} AS BIGINT)),0) AS BIGINT) FROM {table};")
    return int(cur.fetchone()[0]) + 1

def random_fk(cur, table, id_col, fallback=1):
    cur.execute(f"SELECT MIN(TRY_CAST({id_col} AS BIGINT)), MAX(TRY_CAST({id_col} AS BIGINT)) FROM {table};")
    mn, mx = cur.fetchone()
    return fallback if mn is None or mx is None else random.randint(int(mn), int(mx))


In [4]:
#Inserciones por tabla
def insert_cliente(cur):
    i = get_next_id(cur,"clientes","ID_cliente")
    cur.execute("INSERT INTO clientes VALUES (?,?,?,?,?)",
                i, maybe_typo(f"Cliente_{i}"), random.choice(PAISES),
                "correo_invalido" if random.random()<0.05 else f"cliente{i}@mail.com",
                f"+56 9 {random.randint(10000000,99999999)}")

def insert_producto(cur):
    i = get_next_id(cur,"productos","ID_producto")
    cur.execute("INSERT INTO productos VALUES (?,?,?,?,?)",
                i, maybe_typo(f"Producto_{i}"), random.choice(ESPECIES),
                random.choice(PROCESOS if random.random()>0.02 else ["Filet"]),
                random.randint(2000,10000))

def insert_empleado(cur):
    i = get_next_id(cur,"empleados","ID_empleado")
    cur.execute("INSERT INTO empleados VALUES (?,?,?,?,?,?)",
                i, f"Empleado_{i}", random.choice(CARGOS),
                random.choice(PLANTAS), random.choice(TURNOS),
                random.randint(500000,2000000))

def insert_produccion(cur):
    i = get_next_id(cur,"produccion","ID_produccion")
    cur.execute("INSERT INTO produccion VALUES (?,?,?,?,?,?)",
                i,
                random_fk(cur,"productos","ID_producto"),
                random.choice(PLANTAS if random.random()>0.01 else [None]),
                random_fk(cur,"empleados","ID_empleado"),
                (-1 if random.random()<P_NEG_KG else 1)*int(random.normalvariate(1000,250)),
                rand_dt())

def insert_venta(cur):
    i = get_next_id(cur,"ventas","ID_venta")
    precio = None if random.random()<P_NULL_PRICE else random.randint(10000,1000000)
    cur.execute("INSERT INTO ventas VALUES (?,?,?,?,?,?)",
                i, random_fk(cur,"clientes","ID_cliente"),
                random_fk(cur,"productos","ID_producto"),
                random.randint(1,500), precio, rand_dt())

def insert_exportacion(cur):
    i = get_next_id(cur,"exportaciones","ID_exportacion")
    cur.execute("INSERT INTO exportaciones VALUES (?,?,?,?,?,?,?)",
                i, random_fk(cur,"clientes","ID_cliente"),
                random_fk(cur,"productos","ID_producto"),
                int(random.normalvariate(500,200)),
                random.choice(PAISES), random.choice(ESTADOS), rand_dt())

def insert_costo(cur):
    i = get_next_id(cur,"costos","ID_costo")
    energia = -9999 if random.random()<0.02 else int(random.normalvariate(2_000_000,500_000))
    cur.execute("INSERT INTO costos VALUES (?,?,?,?,?,?)",
                i, random.choice(PLANTAS), energia,
                int(random.normalvariate(3_000_000,800_000)),
                int(random.normalvariate(1_000_000,300_000)),
                random.randint(1,12))

def insert_inventario(cur):
    i = get_next_id(cur,"inventario","ID_inventario")
    cur.execute("INSERT INTO inventario(ID_inventario,ID_producto,Stock_kg,Ubicacion) VALUES (?,?,?,?)",
                i, random_fk(cur,"productos","ID_producto"),
                random.randint(0,10000), random.choice(UBICS))


In [5]:
#Sembrar dimensiones (una vez)
with get_conn() as conn:
    cur = conn.cursor()
    for _ in range(50):
        insert_cliente(cur); insert_producto(cur); insert_empleado(cur)
print("Dimensiones sembradas ✔️")


Dimensiones sembradas ✔️


In [6]:
#Stream de inserciones
import time, random
BATCH_MIN, BATCH_MAX = 50, 150
SLEEP_SECONDS = 2
NUM_CICLOS = None   # pon None para infinito (detén con Ctrl+C)

ops = [insert_produccion, insert_venta, insert_exportacion, insert_costo, insert_inventario]

with get_conn() as conn:
    cur = conn.cursor()
    try:
        if NUM_CICLOS is None:
            ciclo = 0
            while True:
                ciclo += 1
                for _ in range(random.randint(BATCH_MIN,BATCH_MAX)):
                    random.choice(ops)(cur)
                if random.random()<0.10: insert_cliente(cur)
                if random.random()<0.08:  insert_producto(cur)
                if random.random()<0.06:  insert_empleado(cur)
                print("Ciclo", ciclo, "ok")
                time.sleep(SLEEP_SECONDS)
        else:
            for c in range(1, NUM_CICLOS+1):
                for _ in range(random.randint(BATCH_MIN,BATCH_MAX)):
                    random.choice(ops)(cur)
                if random.random()<0.10: insert_cliente(cur)
                if random.random()<0.08:  insert_producto(cur)
                if random.random()<0.06:  insert_empleado(cur)
                print("Ciclo", c, "ok")
                time.sleep(SLEEP_SECONDS)
    except KeyboardInterrupt:
        print("Interrumpido por el usuario")


Ciclo 1 ok
Ciclo 2 ok
Ciclo 3 ok
Ciclo 4 ok
Ciclo 5 ok
Ciclo 6 ok
Interrumpido por el usuario


In [9]:
#Verificar conteos
with get_conn() as conn:
    cur = conn.cursor()
    for t in ["clientes","productos","empleados","produccion","ventas","exportaciones","costos","inventario"]:
        cur.execute(f"SELECT COUNT(1) FROM {t};")
        print(t, cur.fetchone()[0])


clientes 2051
productos 550
empleados 3050
produccion 80105
ventas 80109
exportaciones 80116
costos 80114
inventario 10117
